In [3]:
%reload_ext autoreload
%autoreload 2

# Original DTW
    # Look at worst match - based on average distance
        # Randomly sample among worst matches, based on how bad they are
    # Attempt reversing/swapping
        # Choose new GT
         # Add some buffer
         # Choose downstream/upstream baseline points
         # Get distance of downstream point
        # Refill cost matrix
            # Reuse upstream portion
            # Limit downstream portion
        # Traceback
            # Finds new path
            # if better, replace path through window+buffer of original DTW
            # Stroke to Dataloader GT

from forbiddenfruit import curse
import numpy as np

def push_back(self, a):
    self.append(a)
#list.push_back = push_back

def push_back(self, a):
    self.insert(0,a)
    
curse(list, "push_back", push_back)


# DTW each stroke iteratively
# DTW until stroke+1 is chosen
    # stroke + i is the min of stroke/reverse stroke
    # check whether the stroke or reverse stroke does better
INFINITY = 1000

def euclidean_distance(a, b):
    d = 0
    a = np.asarray([a])
    b = np.asarray([b])
    #print(a.shape,b)
    for i in range(a.shape[0]):
        tmp = a[i] - b[i]
        d += tmp * tmp
    return np.sqrt(d)

def d_argmin(a, b, c):
    if a <= b and a <= c:
        return 0
    elif b <= c:
        return 1
    else:
        return 2

def d_min(a, b, c):
    if a < b and a < c:
        return a
    elif b < c:
        return b
    else:
        return c

    
def traceback_partial(cost_mat, ilen, jlen, imin=0, jmin=0):
    cost = 0.0
    i = ilen - 1
    j = jlen - 1
#     cdef vector[int] a
#     cdef vector[int] b
#     a.push_back(i)
#     b.push_back(j)

    a = []
    b = []
    a.append(i)
    b.append(j)
    #cdef int match
    while (i > imin or j > jmin):
        match = d_argmin(cost_mat[i - 1, j - 1], cost_mat[i - 1, j], cost_mat[i, j - 1])
        if match == 0:
            i -= 1
            j -= 1
            cost += cost_mat[i - 1, j - 1]
        elif match == 1:
            i -= 1
            cost += cost_mat[i - 1, j]
        else:
            j -= 1
            cost += cost_mat[i, j - 1]
        a.push_back(i)
        b.push_back(j)
    return a, b, cost

def create_cost_mat_2d(a,b,constraint, dist_func=euclidean_distance):
    cost_mat = np.empty((a.shape[0] + 1, b.shape[0] + 1), dtype=np.float64)
    cost_mat[:] = INFINITY
    cost_mat[0, 0] = 0
    for i in range(1, cost_mat.shape[0]):
        for j in range(max(1, i-constraint), min(cost_mat.shape[1], i+constraint+1)):
            cost_mat[i, j] = dist_func(a[i - 1], b[j - 1]) +\
                d_min(cost_mat[i - 1, j], cost_mat[i, j - 1], cost_mat[i - 1, j - 1])

    return cost_mat[1:, 1:]
    
def traceback(cost_mat, ilen, jlen):
    cost = 0.0
    i = ilen - 1
    j = jlen - 1
#     cdef vector[int] a
#     cdef vector[int] b
#     a.push_back(i)
#     b.push_back(j)

    a = []
    b = []
    a.append(i)
    b.append(j)
    #cdef int match
    while (i > 0 or j > 0):
        match = d_argmin(cost_mat[i - 1, j - 1], cost_mat[i - 1, j], cost_mat[i, j - 1])
        if match == 0:
            i -= 1
            j -= 1
            cost += cost_mat[i - 1, j - 1]
        elif match == 1:
            i -= 1
            cost += cost_mat[i - 1, j]
        else:
            j -= 1
            cost += cost_mat[i, j - 1]
        a.push_back(i)
        b.push_back(j)
    return a, b, cost


np.set_printoptions(suppress=True)
    
x1 = np.array([1,2.0,3])
x2 = np.array([1,1.2,3])
mat = create_cost_mat_2d(x1,x2,constraint=5)
r = traceback(mat, x1.shape[0], x2.shape[0])
print(r)

if False:
    x1 = np.array([1,2.9,2.9,3])
    x2 = np.array([1.1,1.2,1.4,3])
    mat = create_cost_mat_2d(x1,x2,constraint=5)
    print(mat)
    traceback(mat, x1.shape[0], x2.shape[0])

# [0 0 0 1 2 3]
# [0 1 2 3 3 3]



([0, 1, 2], [0, 1, 2], 0.8)


In [4]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")
sys.path.append("../")
sys.path.append("/media/data/GitHub/simple_hwr")
import os
import numpy as np
from hwr_utils import *
from hwr_utils.stroke_plotting import *
from hwr_utils.stroke_recovery import get_number_of_stroke_pts_from_gt
from hwr_utils.stroke_recovery import *
import json
from matplotlib import pyplot as plt

In [6]:
from pydtw import dtw
# Original DTW
    # Look at worst match - based on average distance
        # Randomly sample among worst matches, based on how bad they are
    # Attempt reversing/swapping
        # Choose new GT
         # Add some buffer
         # Choose downstream/upstream baseline points
         # Get distance of downstream point
        # Refill cost matrix
            # Reuse upstream portion
            # Limit downstream portion
        # Traceback
            # Finds new path
            # if better, replace path through window+buffer of original DTW
            # Stroke to Dataloader GT

"""
for i in range(1, cost_mat.shape[0]):
    for j in range(max(1, i-constraint), min(cost_mat.shape[1], i+constraint+1)):
"""
def refill_cost_matrix(a, b, cost_mat, start_a, end_a, start_b, end_b, constraint, dist_func=euclidean_distance):
    # Include some buffer beyond just the strokes being flipped
    # To get improvement, compare the cost at this point before and after
    #cost_mat = np.empty((a.shape[0] + 1, b.shape[0] + 1), dtype=np.float64)
    #cost_mat[:] = INFINITY
    #cost_mat[0, 0] = 0

    #     start_a = max(start_a - 1, 0)
    #     start_b = max(start_b - 1, 0)
    #     end_a = max(end_a - 1, 0)
    #     end_b = max(end_b - 1, 0)
    
    for i in range(start_a+1, end_a+1):
        for j in range(max(start_b+1, i-constraint), min(end_b+1, i+constraint+1)):
            cost_mat[i, j] = dist_func(a[i - 1], b[j - 1]) +\
                d_min(cost_mat[i - 1, j], cost_mat[i, j - 1], cost_mat[i - 1, j - 1])

    return cost_mat[1:, 1:]

gt = np.array(range(36)).reshape(9,4).astype(np.float64)
gt[:,2] = [1,0,0,1,0,1,1,0,0]

preds = [[8, 9, 1, 3], [4, 5, 0, 7], [0, 1, 0, 11], [12, 13, 1, 15], [16, 17, 0, 19], [20, 21, 1, 23], [32, 33, 1, 27], [28, 29, 0, 31], [24, 25, 0, 35]]
preds = [[8, 9, 1, 3], [4, 5, 0, 7], [0, 1, 0, 11], [12, 13, 1, 15], [16, 17, 0, 19], [20, 21, 1, 23], [21, 21, 1, 23], [32, 33, 1, 27], [28, 29, 0, 31], [24, 25, 0, 35]]
preds = [[0, 1, 0, 11], [4, 5, 0, 7], [8, 9, 1, 3], [12, 13, 1, 15], [16, 17, 0, 19], [20, 21, 1, 23], [21, 21, 1, 23], [32, 33, 1, 27], [28, 29, 0, 31], [24, 25, 0, 35]]
preds = np.asarray(preds).astype(np.float64)
print(gt)

# traceback(mat, x1.shape[0], x2.shape[0])
#create_cost_mat_2d
#constrained_dtw2d
CONSTRAINT = 5 
buffer = 0
def get_worst_match(gt, preds, a, b, sos):
    """ Return the stroke number with the worst match
    """
    error = abs(gt[a]-preds[b])**2
    print("error", error)
    for x in np.split(gt, sos)[1:]:
        print(x)
    
    strokes = np.split(error, sos)[1:]

    ## ACTUAL DISTANCE
    if True:
        m = np.sum(np.sum(strokes[-1],axis=1)**.5)

    x = [np.sum(x)  for x in strokes]
    
    print("average mismatch cost", x)
    return np.argmax(x)

# Try reversing the stroke
    # Refill ONLY the stroke of the matrix + end buffer
    # Traceback from end buffer
# These are all GT indices
    # First ROW/COL of cost matrix are NULL!

cost_mat, costr, a, b = dtw.constrained_dtw2d(np.ascontiguousarray(gt[:,:2]),np.ascontiguousarray(preds[:,:2]),constraint=CONSTRAINT)
sos = get_sos_args(gt[:,2], stroke_numbers=False)
worst_match_idx = get_worst_match(gt[:,:2],preds[:,:2],a,b,sos)

# Convert the stroke number to indices in GT
start_idx = sos[worst_match_idx]
start_idx_buffer = max(start_idx-buffer, 0) # double check -1
end_idx = gt.shape[0] if worst_match_idx+1 >= sos.size or sos[worst_match_idx] > gt.size else sos[worst_match_idx+1]
end_idx_buffer = gt.shape[0] if worst_match_idx+1 >= sos.size or sos[worst_match_idx]+buffer > gt.size else sos[worst_match_idx]+buffer
new_gt = gt[end_idx:start_idx_gt:-1, :2]

# Old Cost
if end_idx_buffer:
    alignment_end_idx = np.argmax(a==end_idx_buffer) # first GT point
    old_cost = cost_mat[a[alignment_end_idx],b[alignment_end_idx]] # where we will start the traceback later
else:
    old_cost = cost_mat[a[-1],b[-1]]



[[ 0.  1.  1.  3.]
 [ 4.  5.  0.  7.]
 [ 8.  9.  0. 11.]
 [12. 13.  1. 15.]
 [16. 17.  0. 19.]
 [20. 21.  1. 23.]
 [24. 25.  1. 27.]
 [28. 29.  0. 31.]
 [32. 33.  0. 35.]]
error [[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 9. 16.]
 [16. 16.]
 [ 0.  0.]
 [64. 64.]]
[[0. 1.]
 [4. 5.]
 [8. 9.]]
[[12. 13.]
 [16. 17.]]
[[20. 21.]]
[[24. 25.]
 [28. 29.]
 [32. 33.]]
average mismatch cost [0.0, 0.0, 0.0, 185.0]


In [9]:
np.array([1,2,3])[-1:0:-1]

array([3, 2])

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
def d_min(a, b, c):
    if a < b and a < c:
        return a
    elif b < c:
        return b
    else:
        return c


In [ ]:
from pydtw import *
# from taylor_dtw.custom_dtw import *

x1 = np.array([1,2.9,2.9,3,5,6,7,8])
x2 = np.array([1.1,1.2])

x = constrained_dtw1d(x2,x1,2)
print(x)
x = constrained_dtw1d(x1,x2,2)
print(x)
x = dtw1d(x2,x1)
print(x)
x = dtw1d(x1,x2)
print(x)


mat = create_cost_mat_2d(x1,x2,constraint=5)
print(mat)
print(traceback(mat, len(x1), len(x2)))
mat = create_cost_mat_2d(x2,x1,constraint=5)
print(mat)
print(traceback(mat, len(x2), len(x1)))

In [ ]:
x1 = np.array([1,2.9,2.9,3])
x2 = np.array([1.1,1.2,1.4,3])
mat = create_cost_mat_2d(x1,x2,constraint=50000000)
print(mat)


In [ ]:
import numpy as np
#from pydtw import *
x1 = np.array([1,2.9,2.9,3])
x2 = np.array([1.1,1.2,1.4,3])

#constrained_dtw1d(x1,x2,10)
f, cost, a, b = dtw1d(x1,x2)
print(np.sum(np.abs(x1[a]-x2[b])), cost)
print(a, b)
print(np.asarray(f))

In [ ]:
x1 = np.array([1,2.0,3])
x2 = np.array([1,1.2,3])
mat = create_cost_mat_2d(x1,x2, constraint=10)
dist, cost, a, b = dtw1d(x1, x2, )
print(mat)

In [ ]:
def process(gt, gt_next, pred, str_len, window_size=window):
    """ Match the GT stroke + a few more strokes
        Get last match to GT stroke
        
        
        gt_next: next stroke
    """
    _comb = np.concatenate([gt, gt_next], axis=0)
    #print("combined", _comb, pred)
    f, costr, a, b = constrained_dtw1d(_comb,pred, window_size)
    #print(a,b)
    #print("GT", _comb[a])
    #print("Pred", pred[b])
    dist = np.sum(abs(_comb[a]-pred[b]))
    return dist, a, b
    


def DTW(gt, preds,window=5):
    gt_1d = gt[:,0].copy(order='C')
    preds_1d = preds[:,0].copy(order='C')

    gt_stroke_lens = get_number_of_stroke_pts_from_gt(gt, stroke_numbers=False)
    gt_stroke_lens = np.insert(gt_stroke_lens,0,0) # possibly swap first stroke

    match_a = []
    match_b = []
    # Cython Function that takes exisiting distance matrix and only fills out the last bit

    pos_gt = 0
    pos_pred = 0

    ## FIRST ATTEMPT AT MANUAL DTW MATCH
    for i in range(len(gt_stroke_lens)):
        next_stroke_length = gt_stroke_lens[i+1] if i+1<len(gt_stroke_lens) else 0
        str_len = gt_stroke_lens[i]
        temp_window = min(window, next_stroke_length)

        end_gt, end_pred = pos_gt+str_len, pos_pred+str_len+temp_window
        gt_next = gt_1d[end_gt:end_gt+next_stroke_length]
        gt_stroke = gt_1d[pos_gt:end_gt]
        pred_stroke = preds_1d[pos_pred:end_pred]    
        #print(gt_stroke, gt_next, pred_stroke)
        dist,a,b = process(gt_stroke,gt_next[:temp_window],pred_stroke,str_len,window)  
        

        if i < len(gt_stroke_lens)-1:
            gtr_next=gt_next[::-1][:temp_window].copy(order='C')
            distr,ar,br = process(gt_stroke,gtr_next,pred_stroke,str_len,temp_window)   
            # Invert the next stroke
            #print("LOSS", dist, distr)
            if distr<dist:
                #print("REVERSE REVERSE!")
                gt_1d[end_gt:end_gt+next_stroke_length] = gtr_next
                a,b = ar, br

            #print("A", a)
            #print("B", b)



            if i>0: # first one doesn't save alignment
                last_stroke_idx_a = len(a) - 1 - np.argmax(a[::-1]==str_len) # how far to go in the alignment list
                # -1 because 0 indexing; 10 - np.argmax(np.array([5]*10)[::-1]==5) - 1
                #print("How far to go in alignment list", last_stroke_idx_a)
                pos_pred_update = b[last_stroke_idx_a] # how far the pred matches
                #print("How far the pred matches", pos_pred_update)
                #print("How far the gt matches (by def)", str_len)

                match_a += [x+pos_gt   for x in   a[:last_stroke_idx_a]]
                match_b += [x+pos_pred for x in b[:last_stroke_idx_a]]
                pos_gt += str_len
                pos_pred += pos_pred_update
        else:
            match_a += [x+pos_gt   for x in a]
            match_b += [x+pos_pred for x in b]

#         print("MATCH A:", match_a)
#         print("MATCH B:", match_b)
#         print("NEW POS:", pos_gt, pos_pred)
#     print(gt_1d)


In [ ]:
gt = np.array(range(36)).reshape(9,4).astype(np.float64)
gt[:,2] = [1,0,0,1,0,1,1,0,0]
preds = [[8, 9, 1, 3], [4, 5, 0, 7], [0, 1, 0, 11], [12, 13, 1, 15], [16, 17, 0, 19], [20, 21, 1, 23], [32, 33, 1, 27], [28, 29, 0, 31], [24, 25, 0, 35]]
preds = [[8, 9, 1, 3], [4, 5, 0, 7], [0, 1, 0, 11], [12, 13, 1, 15], [16, 17, 0, 19], [20, 21, 1, 23], [21, 21, 1, 23], [32, 33, 1, 27], [28, 29, 0, 31], [24, 25, 0, 35]]
preds = np.asarray(preds).astype(np.float64)

pos_gt = 0
pos_pred = 0

print(gt)
print(gt_stroke_lens)
print(preds)

DTW(gt, preds, window = 5)

In [ ]:
def find_worst_strokes(gt, pred, a, b):

    
    
f, cost, a, b = constrained_dtw1d(gt,pred, window=20)


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
max_width = 8

t = []
for i in range(10000):
    #x = min(max(int(abs(np.random.exponential()) * (max_width) * .2 + 2), 1), max_width)
    x = min(max(int(abs(np.random.randn()) * (max_width - 1) * .5 + 1), 1),max_width)
    #x = min(max(int(abs(np.random.beta(2,5)) * max_width + 1), 1),max_width)

    t.append(x)

plt.hist(t)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
max_width = 8

t = []
for i in range(10000):
    #x = min(max(int(abs(np.random.exponential()) * (max_width) * .2 + 2), 1), max_width)
    #x = min(max(int(abs(np.random.randn()) * (max_width - 1) * .5 + 1), 1),max_width)
    x = min(max(int(abs(np.random.beta(2,4)) * max_width + .8), 1),max_width)

    t.append(x)

plt.hist(t)